In [1]:
import os
import pdfplumber
import pandas as pd
import numpy as np
import re

# libro = pd.read_excel('file_informeMensualREC - 2025-12-02T162142.258.xlsx', skiprows=5)
# libro

In [ ]:
ruta_carpeta = 'boletas_pdf/' 
for file in os.listdir(ruta_carpeta):
    ruta_pdf = ruta_carpeta + file

    with pdfplumber.open(ruta_pdf) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
        lines = text.split('\n')
        

    id_boleta = 'N ° '
    for linea in lines:
        if id_boleta in linea:
            id_boleta_valor = linea.split(id_boleta)[1].strip()
            print(f'ID Boleta: {id_boleta_valor}')

    id_rut = 'RUT: '
    for linea in lines:
        if id_rut in linea:
            id_rut_valor = linea.split(id_rut)[1].strip()
            print(f'RUT: {id_rut_valor}')


    ini = 'Por atención profesional:'
    fin = 'Total Honorarios'
    flag = False
    df_glosa = pd.DataFrame(columns=['Numero Boleta', 'RUT', 'Glosa', 'Valor'])

    for linea in lines:
        if ini in linea:
            flag = True
            continue
        
        if fin in linea:
            flag = False
            continue
        
        if flag == True:
            glosa = linea.strip()
            
            valor_ = glosa.split()[-1]
            if '.' in valor_:
                valor = np.int64(valor_.replace('.', ''))
                glosa = glosa.replace(valor_, '')
            else:
                valor = None

            df_glosa = pd.concat([df_glosa, pd.DataFrame({'Numero Boleta': [id_boleta_valor], 'RUT': [id_rut_valor], 'Glosa': [glosa], 'Valor':[valor]})], ignore_index=True)

    df_glosa = df_glosa.reset_index(drop=True)
    for ix in df_glosa.index:
        if df_glosa.at[ix, 'Valor'] == None:
            df_glosa.at[ix-1, 'Glosa'] = df_glosa.at[ix-1, 'Glosa'] + ' ' + df_glosa.at[ix, 'Glosa']
            df_glosa = df_glosa.drop(ix)

df_glosa.to_excel('glosas_boleta.xlsx', index=False)


['BOLETA DE HONORARIOS', 'EDUARDO JAVIER DIEZ MORELLO ELECTRONICA', 'N ° 152666', 'RUT: 5.248.026−4', 'GIRO(S): DETALLE', 'LUIS THAYER OJEDA 359 PROVIDENCIA SANTIAGO , PROVIDENCIA', 'Fecha: 21 de Noviembre de 2025', 'Señor(es): EMPRESA DE BUSES HUALPEN LTDA Rut: 84.794.200− 2', 'Domicilio: MIGUEL CLARO 368 101 PROVIDENCIA SANTIAGO', 'Por atención profesional:', 'COPIA ADICIONAL 13.500', 'Total Honorarios: $: 13.500', 'Fecha / Hora Emisión: 21/11/2025 13:16', '052480261526665BEEA8', 'Res. Ex. N° 83 de 30/08/2004', 'Verifique este documento en www.sii.cl', 'El contribuyente emisor de esta boleta debe declarar y enterar el PPM de Segunda Categoría correspondiente al porcentaje definido.', '11202512041717', 'Fecha / Hora Impresión: 04/12/2025 17:17', '1']
ID Boleta: 152666
RUT: 5.248.026−4
['BOLETA DE HONORARIOS', 'PATRICIA JACQUELINE JANA TRONCOSO ELECTRONICA', 'N ° 278', 'RUT: 10.689.553−8', 'GIRO(S): OTRAS ACTIVIDADES DE SERVICIOS PERSONALES N.C.P.,', 'MODISTA−LAVANDERA', 'LARQUI 9238 V

In [ ]:
# --- Ejecución ---
# Reemplaza con tu archivo real
archivo = "TMBCOT_ConsultaBoletaPdf (8).cgi.pdf" 

try:
    resultado = extraer_datos_boleta(archivo)
    
    print("-" * 30)
    print("RESULTADOS DE EXTRACCIÓN")
    print("-" * 30)
    for k, v in resultado.items():
        print(f"{k}: {v}")
        
except Exception as e:
    print(f"Error al procesar: {e}")

In [ ]:

# --- Ejecución ---
# Reemplaza con tu archivo real
archivo = "TMBCOT_ConsultaBoletaPdf (8).cgi.pdf" 

try:
    resultado = extraer_datos_boleta(archivo)
    
    print("-" * 30)
    print("RESULTADOS DE EXTRACCIÓN")
    print("-" * 30)
    for k, v in resultado.items():
        print(f"{k}: {v}")
        
except Exception as e:
    print(f"Error al procesar: {e}")